In [296]:
import random
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod

In [411]:
class randomGenerator(ABC):
    @abstractmethod
    def generate():
        pass
    
    
    def mean(self, nb_sim):
        return sum(self.generate())/nb_sim
    
    
    def variance(self, nb_sim):
        return sum(self.generate()**2)/nb_sim - (sum(self.generate())/nb_sim)**2

class uniformGenerator(randomGenerator):
    @abstractmethod
    def generate():
        pass


class pseudoGenerator(uniformGenerator):
    def __init__(self,Seed):
            self.Seed=Seed
    def mean(self, nb_sim):
        return self.Seed/nb_sim
    def variance(self, nb_sim):
        return self.Seed/nb_sim
    def generate():
         pass
    

class linearCongruential(pseudoGenerator):
    def __init__(self,Multiplier,Modulus,Increment,pseudoGenerator:pseudoGenerator) -> None:
            self.Seed = pseudoGenerator.Seed
            self.Multiplier = Multiplier
            self.Modulus = Modulus
            self.Increment = Increment
            self.x0 = self.Seed
    def generate(self):
        self.x0 = (self.Multiplier * self.x0 + self.Increment) % self.Modulus
        return self.x0/self.Modulus
        

    def get_x(self):
        return self.x0
    
class ecuyerCombined(pseudoGenerator):
    def __init__(self, gen:pseudoGenerator) -> None:
        self.seed = gen.Seed
        self.ecu = 0.0
        self.gen1 = pseudoGenerator(Seed=(self.seed+1))
        self.gen2 = pseudoGenerator(Seed=17*(self.seed+1)+41)
        self.l1 = linearCongruential(40014, 2014704830563, 0, self.gen1)
        self.l2 = linearCongruential(40692, 2014704830399, 0, self.gen2)
        
    def generate(self):
        
        self.l1.generate()
        self.l2.generate()
        
        self.ecu = (self.l1.get_x()-self.l2.get_x()) % 2014704830563
        
        if self.ecu > 0:
            return self.ecu/2014704830563 #correct d'utiliser les deux modulus comme ça ?
        else:
            return 2014704830562 / 2014704830563
        
    
    def get_ecu(self):
        return self.ecu
    

class discreteGenerator(randomGenerator):
    def generate():
        pass
    
class headTail(discreteGenerator):
    def __init__(self, ecu: ecuyerCombined) -> None:
        self.ecu = ecu
    
    def generate(self):
        ecu = self.ecu.generate()
        
        if ecu <= 1/2:
            return "Head"
        else:
            return "Tail"
class bernoulli(discreteGenerator):
    def __init__(self, p, ecu: ecuyerCombined) -> None:
        self.p = p
        self.ecu = ecu
    
    def generate(self):
        ecu = self.ecu.generate()
        
        if ecu <= self.p:
            return 1
        else:
            return 0
        
class binomial(discreteGenerator):
    def __init__(self, n, p, ecu: ecuyerCombined) -> None:
        self.n = n
        self.p = p
        self.ecu = ecu
    
    def generate(self):
        sim = []
        for i in range(self.n):
            ber = bernoulli(self.p, self.ecu).generate()
            sim.append(ber)
        return sum(sim)

class finiteSet(discreteGenerator):
    def __init__(self, n, k: list, p, ecu: ecuyerCombined) -> None:
        self.n = n
        self.p = p
        self.k = k
        self.ecu = ecu
    
    def generate(self):
        bin = binomial(self.n, self.p, self.ecu).generate()
        
        self.k.sort()
        for id, i in enumerate(self.k):
            if bin <= i*self.n:
                return id

class poisson(discreteGenerator):
    def __init__(self) -> None: #comment faire pour poisson ?
        pass
    


Countinuous RVs

In [472]:

class countinuousGenerator(randomGenerator):
    def generate():
        pass

class exponential(countinuousGenerator):
    def __init__(self, lmda, ecu: ecuyerCombined) -> None:
        self.lmda = lmda
        self.ecu = ecu
    
    def generate(self):
        pass           

class expoInverse(exponential):
    def __init__(self, lmda, ecu: ecuyerCombined) -> None:
        super().__init__(lmda, ecu)
        
    def generate(self):
        ecu = self.ecu.generate()
        return - np.log(ecu) / self.lmda

class expoRejectionSampling(exponential): 
    #doit rentrer les parametres de la fonction ou ça doit etre calculé à l'interieur de la fonction?
    # comment on faire pour la fontion ? 
    
    def __init__(self, lmda, ecu: ecuyerCombined, ecu2: ecuyerCombined, a, b, M) -> None:
        super().__init__(lmda, ecu)
        self.ecu2 = ecu2
        self.a = a
        self.b = b
        self.M = M
    
    def generate(self):
        u1 = self.ecu.generate()
        u2 = self.ecu2.generate()
        
        x = self.a + (self.b - self.a) * u1
        y = self.M * u2
        
        


    def f(x,a,b):
        return 6*x*(1-x) if x>=a and x<=b else 0
    
    def M(x,a,b):
        m = f(a,a,b)
        for i in range(a,b):
            if f(i,a,b) > m:
                m = f(i,a,b)


class normal(countinuousGenerator):
    def __init__(self, gen: pseudoGenerator) -> None:
        self.gen = gen
    
    def generate():
        pass
    
class boxMuller(normal):
    def __init__(self, gen: pseudoGenerator) -> None:
        super().__init__(gen)
    
    def generate(self):
        generated_lin = []
        for i in range(2*2):
            generated_lin.append(linearCongruential(Multiplier=17*(i+1), Increment=100-i*10, constant=43*i, Modulus=2**31, pseudoGenerator=self.gen))
        
        ecu1 = ecuyerCombined(generated_lin[0],generated_lin[1])
        ecu2 = ecuyerCombined(generated_lin[2],generated_lin[3])
        
        R = np.sqrt(-2*np.log(ecu1.generate()))
        theta = 2 * np.pi * ecu2.generate()
        
        x = R * np.cos(theta)
        y = R * np.sin(theta)
        
        
        
class centralLimit(normal):
    def __init__(self, gen: pseudoGenerator) -> None:
        self.gen = gen
        self.ecu = ecuyerCombined(self.gen)
    
    def generate(self):
        res = 0
        for i in range(12):
            res += self.ecu.generate()
        
        return res - 6




In [502]:
generator1 = pseudoGenerator(Seed=29)
generator2 = pseudoGenerator(Seed=51)
generator3 = pseudoGenerator(Seed=62)
generator4 = pseudoGenerator(Seed=12)

#Generator1 = linearCongruential(2147483562, 40014, 0, 2147483563, pseudoGenerator=generator1)
#Generator2 = linearCongruential(2147483398, 40692, 0, 2147483399, pseudoGenerator=generator1)

l1 = linearCongruential(Multiplier=2147483563, Increment=40014, Modulus=2**18, pseudoGenerator=generator1)
l2 = linearCongruential(Multiplier=2147483562, Increment=40014, Modulus=2**18, pseudoGenerator=generator2)

l3 = linearCongruential(Multiplier=21, Increment=24, Modulus=2**31, pseudoGenerator=generator3)
l4 = linearCongruential(Multiplier=37, Increment=71, Modulus=2**31, pseudoGenerator=generator4)

ecu = ecuyerCombined(generator1)
#ecu2 = ecuyerCombined(l3,l4)
HT = headTail(ecu)
Ber = bernoulli(0.7, ecu)
Bin = binomial(1000, 0.7,ecu)
finset = finiteSet(1000,[0, 0.1, 0.4, 0.5, 0.9, 1], 0.48, ecu)
exp_c = exponential(2, ecu)
#exp_inv = expoInverse(exp_c)

central = centralLimit(generator1)



In [503]:
l1.generate()
ecu.generate()
HT.generate()
Ber.generate()
Bin.generate()
finset.generate()
#exp_inv.generate()

central.generate()


1.3211678669669853

In [74]:
def fact(n):
    if n == 1:
        return 1
    else:
        return n * fact(n-1)

720

In [69]:

p = 0
s = 0
lambd = 1 
for i in range(100):
    p_k = np.exp(-lambd) * lambd**i / fact(i)
    p_k = (lambd/(i+1))*p_k
    s+=p
    print(p , s)

p0 = 

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
